## Working with SQL Databases

There are a great number of python modules that provide functionalities to work with databases of all variants and flavors. For a MySQL database, we may form a connection using `pymysql` or one of many other alternatives:

```python
import pymysql
conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    db=database)
```

We can then use `pd.read_sql_query()`, passing in the connection:

```python
sales = pd.read_sql_query("SELECT * FROM sales", conn)
```

Under the hood, pandas uses [`SQLAlchemy`](https://github.com/onlyphantom/sqlalchemy-tutorial) so any database supported by that library will work. This isn't something you need to worry about at this stage of your learning journey, but for the sake for practice, let's also see how a connection URI for a SQLite database looks like:

In [9]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../datasets/loan_small.db")

loans = pd.read_sql_query("SELECT * FROM records", conn)
loans.head()

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,1077501,2011,01/12/2011,1012015,10.0,RENT,1,Low,24000,1,...,0,10.65,B,2,27.65,5861.071414,5000.0,0.0,162.87,munster
1,1077430,2011,01/12/2011,1042013,0.5,RENT,1,Low,30000,1,...,1,15.27,C,3,1.0,1008.71,456.46,117.08,59.83,leinster
2,1077175,2011,01/12/2011,1062014,10.0,RENT,1,Low,12252,1,...,0,15.96,C,3,8.72,3003.653644,2400.0,0.0,84.33,cannught
3,1076863,2011,01/12/2011,1012015,10.0,RENT,1,Low,49200,1,...,0,13.49,C,3,20.0,12226.30221,10000.0,0.0,339.31,ulster
4,1075358,2011,01/12/2011,1012016,1.0,RENT,1,Low,80000,1,...,0,12.69,B,2,17.94,3242.17,2233.1,0.0,67.79,ulster


In the above command, we asked for all columns of a table to be returned to us through the SELECT * command. Well, columns of which table? That would be tables. Together they form an SQL query:

```sql
SELECT * FROM albums
```

In the following cell, we use a similar SQL query with an additional `LIMIT` statement to limit the output to the first 5 records (rows). However, notice that we also set `index_col` so the specified column is recognized as the index:

In [10]:
df = pd.read_sql_query(
    "SELECT * FROM records LIMIT 500",
    conn,
    index_col="id",
)

df.head()

,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,loan_amount,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
id,,,,,,,,,,,,,,,,,,,,,
1077501,2011,01/12/2011,1012015,10.0,RENT,1,Low,24000,1,5000,...,0,10.65,B,2,27.65,5861.071414,5000.0,0.0,162.87,munster
1077430,2011,01/12/2011,1042013,0.5,RENT,1,Low,30000,1,2500,...,1,15.27,C,3,1.0,1008.71,456.46,117.08,59.83,leinster
1077175,2011,01/12/2011,1062014,10.0,RENT,1,Low,12252,1,2400,...,0,15.96,C,3,8.72,3003.653644,2400.0,0.0,84.33,cannught
1076863,2011,01/12/2011,1012015,10.0,RENT,1,Low,49200,1,10000,...,0,13.49,C,3,20.0,12226.30221,10000.0,0.0,339.31,ulster
1075358,2011,01/12/2011,1012016,1.0,RENT,1,Low,80000,1,3000,...,0,12.69,B,2,17.94,3242.17,2233.1,0.0,67.79,ulster


In [11]:
pd.crosstab(df["income_category"], df["home_ownership"])

home_ownership,MORTGAGE,OWN,RENT
income_category,,,
High,1,0,2
Low,137,37,283
Medium,26,2,12


In [29]:
df = pd.read_sql_query(
    "SELECT * FROM records",
    conn,
    index_col="id",
)

df["loan_amount"] = df["loan_amount"].astype(float)

df.groupby(["income_category", "home_ownership"])["loan_amount"].mean()

income_category  home_ownership
High             MORTGAGE          22821.651786
                 OWN               24211.111111
                 RENT              20857.017544
Low              MORTGAGE          12392.373758
                 OTHER             16000.000000
                 OWN               10458.314649
                 RENT              10077.504159
Medium           MORTGAGE          17988.579605
                 OWN               15867.142857
                 RENT              16795.787402
Name: loan_amount, dtype: float64

Notice that common aggregation functions provided by `pandas` have an equivalent in SQL. For example, the `mean()` function in `pandas` is equivalent to the `AVG()` function in SQL. The `sum()` function in `pandas` is equivalent to the `SUM()` function in SQL.

-  `count()` function in `pandas` is equivalent to the `COUNT()` function in SQL. 
-  `min()` and `max()` functions in `pandas` have equivalents: `MIN()` and `MAX()` function in SQL.
- `std()` function in `pandas` is equivalent to the `STDDEV()` SQL function while the `var()` function in `pandas` maps to `VAR()` function

In [31]:
query = """
  SELECT income_category, home_ownership, AVG(loan_amount) AS avg_loan_amount
  FROM records
  GROUP BY income_category, home_ownership
"""

df = pd.read_sql_query(query, 
    con=conn, 
)

df.head(10)

,income_category,home_ownership,avg_loan_amount
0,High,MORTGAGE,22821.651786
1,High,OWN,24211.111111
2,High,RENT,20857.017544
3,Low,MORTGAGE,12392.373758
4,Low,OTHER,16000.000000
5,Low,OWN,10458.314649
6,Low,RENT,10077.504159
7,Medium,MORTGAGE,17988.579605
8,Medium,OWN,15867.142857
9,Medium,RENT,16795.787402


Another interesting observation is how much more efficient SQL is at handling large datasets. This is because SQL is optimized for querying and managing large datasets. When the data doesn't fit into your computer's memory ("RAM"), the SQL database provides a way to query the data without having to front-load all of it into memory. This is why SQL is the go-to tool for data engineers and data scientists when working with large datasets.